In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib
import os
from IPython.display import display
matplotlib.use('Agg')
%matplotlib inline
import pandas as pd
pd.set_option('precision', 4)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from gather_metrics import *
def get_ranked_by_means_df(gather_df, metric_list):
    means_df = gather_df.xs(['test_metrics', 'Folds Mean'], level=['set','fold'])
    ranked_df = pd.DataFrame(data=np.zeros(shape=means_df.shape), columns=metric_list, dtype=str)
    for m, i in zip(metric_list, range(len(metric_list))):
        metric_df = means_df[m]
        sorted_df = metric_df.sort_values(ascending=False)
        mscore_list = sorted_df.tolist()
        models_ranked = sorted_df.index.tolist()
        m_order_mscore_list = [str(m) + ", " + str(round(r,4)) for m, r in zip(models_ranked, mscore_list)]
        ranked_df.iloc[:,i] = m_order_mscore_list
    return ranked_df

# load stage PS
model_directory = '../../../job_results_small/'
class_dirs = [model_directory+'/random_forest/stage_2/',
              model_directory+'/irv/stage_2/',
              model_directory+'/neural_networks/stage_2/',
              model_directory+'/docking/stage_2/',
              model_directory+'/baseline/stage_2/']
gather_df_3 = gather_dir_metrics(class_dirs[0], 1)     
gather_df_3 = gather_df_3.append(gather_dir_metrics(class_dirs[1], 1))   
gather_df_3 = gather_df_3.append(gather_dir_metrics(class_dirs[2], 1))   
gather_df_3 = gather_df_3.append(gather_dir_metrics(class_dirs[3], 1))
gather_df_3 = gather_df_3.append(gather_dir_metrics(class_dirs[4], 1))

col_indices = list(range(10)) + list(range(15, 20)) + list(range(25, 105)) + list(range(145, 149)) + list(range(150, 183))
col_indices = [i for i in col_indices if 'Mean' not in gather_df_3.columns[i] and 'Median' not in gather_df_3.columns[i]]
col_indices = [i for i in col_indices if 'PriA-SSB AS' in gather_df_3.columns[i]]
metric_names = list(gather_df_3.columns.values[col_indices])

gather_df_3 = gather_df_3[metric_names]
gather_df_3 = gather_df_3.dropna(axis=0)

gather_df_ps = gather_df_3

In [2]:
# load stage CV
model_directory = '../../../job_results_small/'
class_dirs = [model_directory+'/random_forest/stage_1/',
              model_directory+'/irv/stage_1/',
              model_directory+'/neural_networks/stage_1/',
              model_directory+'/docking/stage_1/']
k = 5
gather_df_2 = gather_dir_metrics(class_dirs[0], k)     
gather_df_2 = gather_df_2.append(gather_dir_metrics(class_dirs[1], k))   
gather_df_2 = gather_df_2.append(gather_dir_metrics(class_dirs[2], 20))
gather_df_2 = gather_df_2.append(gather_dir_metrics(class_dirs[3], k))
gather_df_2 = gather_df_2[metric_names]
gather_df_2 = gather_df_2.dropna(axis=0)

gather_df_cv = gather_df_2

#setup comparison matrix and overlaps
dtk_dict = dtk_multi_metrics(gather_df_cv, list(range(len(col_indices))))
dtk_analysis_dict = analyze_dtk_dict(dtk_dict)
mm_comp_dicts = get_mean_median_comps(gather_df_cv, list(range(len(col_indices))))
agg_comp_dict = get_agg_comp(mm_comp_dicts, dtk_analysis_dict, mean_w=1, median_w=0, tukey_w=1)

---
## Compute Best Models for Each Metric Based on Mean VS DTK+Mean

In [3]:
ps_250_hits = gather_df_ps.xs(['test_metrics', 'fold 0'], level=['set','fold'])['n_hits_250 PriA-SSB AS']
simple_ranked_cv = get_ranked_by_means_df(gather_df_cv, metric_names)
complex_ranked_cv = get_model_ordering(agg_comp_dict, metric_names)
oracle_pick = ps_250_hits.sort_values(ascending=False).iloc[0]

top_n = 1
req_metrics = ['ROC AUC PriA-SSB AS', 'BEDROC AUC PriA-SSB AS','PR auc.integral PriA-SSB AS', 
               'NEF AUC PriA-SSB AS', 'NEF_1 % PriA-SSB AS', 'EF_1 % PriA-SSB AS']

simple_top_n = simple_ranked_cv[req_metrics].iloc[:top_n,:]
complex_top_n = complex_ranked_cv[req_metrics].iloc[:top_n,:]

### Ranking by Folds Mean Only

In [4]:
display('Ranking by Means. Shows model-mean pairs.', simple_ranked_cv[req_metrics])

'Ranking by Means. Shows model-mean pairs.'

,ROC AUC PriA-SSB AS,BEDROC AUC PriA-SSB AS,PR auc.integral PriA-SSB AS,NEF AUC PriA-SSB AS,NEF_1 % PriA-SSB AS,EF_1 % PriA-SSB AS
0,"RandomForest_d, 0.9134","RandomForest_h, 0.7572","RandomForest_g, 0.217","RandomForest_h, 0.772","RandomForest_h, 0.6467","RandomForest_h, 64.6667"
1,"RandomForest_e, 0.9038","RandomForest_g, 0.7554","RandomForest_h, 0.2147","RandomForest_g, 0.7642","RandomForest_g, 0.6467","RandomForest_g, 64.6667"
2,"RandomForest_h, 0.9012","RandomForest_d, 0.7453","RandomForest_e, 0.2037","RandomForest_d, 0.7578","RandomForest_d, 0.5842","RandomForest_d, 58.4167"
3,"RandomForest_g, 0.8962","RandomForest_e, 0.7418","RandomForest_d, 0.2037","RandomForest_e, 0.7529","RandomForest_e, 0.5842","RandomForest_e, 58.4167"
4,"RandomForest_c, 0.8882","RandomForest_c, 0.7215","RandomForest_b, 0.2008","RandomForest_b, 0.7428","RandomForest_a, 0.5708","RandomForest_a, 57.0833"
5,"RandomForest_a, 0.886","RandomForest_b, 0.719","RandomForest_c, 0.197","RandomForest_c, 0.7381","RandomForest_c, 0.5583","RandomForest_c, 55.8333"
6,"RandomForest_b, 0.8848","RandomForest_a, 0.712","RandomForest_a, 0.196","RandomForest_a, 0.7346","RandomForest_b, 0.5583","RandomForest_b, 55.8333"
7,"SingleRegression_b, 0.8634","IRV_e, 0.6878","IRV_d, 0.1653","IRV_e, 0.699","IRV_c, 0.545","IRV_c, 54.5"
8,"IRV_e, 0.8489","IRV_d, 0.6352","IRV_b, 0.1636","RandomForest_f, 0.6545","IRV_d, 0.545","IRV_d, 54.5"
9,"MultiClassification_b, 0.8475","RandomForest_f, 0.6338","IRV_c, 0.149","IRV_d, 0.6276","IRV_b, 0.5317","IRV_b, 53.1667"


### Ranking by DTK+Means

In [5]:
display('Ranking by DTK+Means. Shows model-rank pairs.', complex_ranked_cv[req_metrics])

'Ranking by DTK+Means. Shows model-rank pairs.'

,ROC AUC PriA-SSB AS,BEDROC AUC PriA-SSB AS,PR auc.integral PriA-SSB AS,NEF AUC PriA-SSB AS,NEF_1 % PriA-SSB AS,EF_1 % PriA-SSB AS
0,"RandomForest_d, 1.0","RandomForest_h, 1.0","SingleClassification_b, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0"
1,"RandomForest_e, 2.0","RandomForest_g, 2.0","SingleClassification_a, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0"
2,"RandomForest_h, 3.0","RandomForest_d, 3.0","MultiClassification_a, 3.0","RandomForest_d, 3.0","RandomForest_d, 3.0","RandomForest_d, 3.0"
3,"RandomForest_g, 4.0","RandomForest_e, 4.0","RandomForest_g, 4.0","RandomForest_e, 4.0","RandomForest_e, 3.0","RandomForest_e, 3.0"
4,"RandomForest_c, 5.0","RandomForest_c, 5.0","RandomForest_h, 5.0","RandomForest_b, 5.0","RandomForest_a, 5.0","RandomForest_a, 5.0"
5,"RandomForest_a, 6.0","RandomForest_b, 6.0","RandomForest_e, 6.0","RandomForest_c, 6.0","RandomForest_b, 6.0","RandomForest_b, 6.0"
6,"RandomForest_b, 7.0","RandomForest_a, 7.0","RandomForest_d, 7.0","RandomForest_a, 7.0","RandomForest_c, 6.0","RandomForest_c, 6.0"
7,"SingleRegression_b, 8.0","IRV_e, 8.0","RandomForest_b, 8.0","IRV_e, 8.0","IRV_c, 8.0","IRV_c, 8.0"
8,"MultiClassification_b, 9.0","IRV_d, 9.0","RandomForest_c, 9.0","RandomForest_f, 9.0","IRV_d, 8.0","IRV_d, 8.0"
9,"IRV_e, 10.0","RandomForest_f, 10.0","RandomForest_a, 10.0","IRV_d, 10.0","IRV_b, 10.0","IRV_b, 10.0"


---
## Regret of best model

In [6]:
simple_regret = simple_top_n.copy()
complex_regret = complex_top_n.copy()

for m in req_metrics:
    simple_model = simple_regret[m].iloc[0]
    simple_model = simple_model[:simple_model.index(',')]
    simple_top_n[m].iloc[0] = simple_model
    complex_model = complex_regret[m].iloc[0]
    complex_model = complex_model[:complex_model.index(',')]
    complex_top_n[m].iloc[0] = complex_model
    
    simple_regret.loc[0,m] = oracle_pick - ps_250_hits.loc[simple_model]
    complex_regret.loc[0,m] = oracle_pick - ps_250_hits.loc[complex_model]

In [7]:
final_df = pd.merge(simple_top_n.T, simple_regret.T, left_index=True, right_index=True)
final_df = pd.merge(final_df, complex_top_n.T, left_index=True, right_index=True)
final_df = pd.merge(final_df, complex_regret.T, left_index=True, right_index=True)
columns = [('Best by Mean','Model'),('Best by Mean','Regret'),('Best by DTK+Mean','Model'),('Best by DTK+Mean','Regret')]
final_df.columns=pd.MultiIndex.from_tuples(columns)
final_df.columns.name = 'Metric'
final_df.index = [s.replace(' PriA-SSB AS', '') for s in final_df.index]

display(final_df)

Best by Mean               Best by DTK+Mean       
                          Model Regret                   Model Regret
ROC AUC          RandomForest_d      3          RandomForest_d      3
BEDROC AUC       RandomForest_h      0          RandomForest_h      0
PR auc.integral  RandomForest_g      1  SingleClassification_b      8
NEF AUC          RandomForest_h      0          RandomForest_h      0
NEF_1 %          RandomForest_h      0          RandomForest_h      0
EF_1 %           RandomForest_h      0          RandomForest_h      0